<a href="https://colab.research.google.com/github/ritwiks9635/ChatGPT.openai/blob/main/Character_level_text_generation_with_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Character-level text generation with LSTM**

## Introduction

This example demonstrates how to use a LSTM model to generate
text character-by-character.

At least 20 epochs are required before the generated text
starts sounding locally coherent.

It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.

If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.

In [ ]:
import io
import keras
import random
import numpy as np
from keras import layers

## Prepare the data

In [ ]:
path = keras.utils.get_file(
    "nietzsche.txt",
    origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")

with io.open(path, encoding = "utf-8") as f:
    text = f.read().lower()
text = text.replace("\n", " ")
print("Corpus length ::", len(text))

chars = sorted(list(set(text)))
print("Total chars:", len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i : i + maxlen])
    next_chars.append(text[i + maxlen])
print("Number of sequences:", len(sentences))

x = np.zeros((len(sentences), maxlen, len(chars)), dtype="bool")
y = np.zeros((len(sentences), len(chars)), dtype="bool")
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

600901/600901 [==============================] - 0s 0us/step
Corpus length :: 600893
Total chars: 56
Number of sequences: 200285


## Build the model: a single LSTM layer

In [ ]:
model = keras.Sequential(
    [
        layers.Input(shape = (maxlen, len(chars))),
        layers.LSTM(128),
        layers.Dense(len(chars), activation = "softmax")
    ])

optimizer = keras.optimizers.RMSprop(learning_rate = 0.01)
model.compile(optimizer = optimizer, loss = "categorical_crossentropy")

## Prepare the text sampling function

In [ ]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

## Train the model

In [ ]:
epochs = 40
batch_size = 128

for epoch in range(epochs):
    model.fit(x, y, batch_size=batch_size, epochs=1)
    print()
    print("Generating text after epoch: %d" % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print("...Diversity:", diversity)

        generated = ""
        sentence = text[start_index : start_index + maxlen]
        print('...Generating with seed: "' + sentence + '"')

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.0
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            sentence = sentence[1:] + next_char
            generated += next_char

        print("...Generated: ", generated)
        print("-")

1565/1565 [==============================] - 11s 5ms/step - loss: 1.9911

Generating text after epoch: 0
...Diversity: 0.2
...Generating with seed: "why should we still punish? punishment i"
...Generated:  n the still to the strenger and to the stepented and as the stepente of the strenger to the still and the still as the stronger and the still and the still as the still as the strenger and to the still to the stappess to an the strenger as the strenge of the stronger to the steppent to the possed the stemparing and and the sten as the will of the still as the self and the strenger and the still an
-
...Diversity: 0.5
...Generating with seed: "why should we still punish? punishment i"
...Generated:  s spetion of the freed for youthing of pertact cires as the self self--an as the say to the stand work an as the such to the stening as a pressente to the posention to the const in that the still herely, it is sense of work mean it not as the man more of the spirit to the discopling as the